In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
from tensorflow.keras import layers
import string
from IPython.display import clear_output

2025-11-12 05:32:37.557111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762925557.747910      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762925557.798173      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
BATCH_SIZE = 128
NUM_HEADS = 12
NUM_BLOCKS = 2
EMBED_DIM = 384
DENSE_DIM = 1536
DROPOUT_RATE = 0.3
CHUNK_LENGTH = 256

In [3]:
df = pd.read_csv('/kaggle/input/the-bards-best-a-character-modeling-dataset/train.csv')

In [4]:
text = df.values[0][0]
text = re.sub(r'\s+', ' ', str(text)).strip()

In [5]:
import re

words = text.split()

# Count unique words
unique_words = set(words)
print(f"Total words: {len(words)}")
print(f"Unique words: {len(unique_words)}")


Total words: 182499
Unique words: 23841


In [6]:
def chunk_text_by_words(text, max_words, stride=None):
    words = text.split()
    if stride is None:
        stride = max_words // 2
    chunks = []
    for i in range(0, len(words) - max_words, stride):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

In [7]:
chunks = chunk_text_by_words(text, CHUNK_LENGTH+1, 30)

In [8]:
print(np.shape(chunks))

(6075,)


In [9]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 12050
sequence_length = CHUNK_LENGTH+1

vectorizer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization
)

vectorizer.adapt(chunks)

I0000 00:00:1762925569.976860      39 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [10]:
vocab = vectorizer.get_vocabulary()
print("Total unique tokens in vocabulary:", len(vocab))

Total unique tokens in vocabulary: 12050


In [11]:
def make_dataset(chunks):
    tokens = vectorizer(chunks)
    tokens_inp = tokens[:,:CHUNK_LENGTH]
    tokens_out = tokens[:,1:]
    ds = tf.data.Dataset.from_tensor_slices((tokens_inp,tokens_out))
    ds = ds.batch(BATCH_SIZE)
    ds = ds.shuffle(1024).prefetch(16).cache()
    return ds

In [12]:
ds = make_dataset(chunks)

In [13]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim):
        super().__init__()
        self.positional_embedding = tf.keras.layers.Embedding(input_dim = sequence_length, output_dim = output_dim, mask_zero=False)
        self.token_embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim= output_dim, mask_zero=True)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.positional_embedding(positions)
        return embedded_tokens + embedded_positions

In [14]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, num_heads, embed_dim, dense_dim, dropout_rate):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                           key_dim=embed_dim//num_heads)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense_proj = tf.keras.models.Sequential([
            tf.keras.layers.Dense(dense_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    def call(self, inputs):
        attn_out = self.attention(query=inputs,
                            key=inputs,
                            value=inputs,
                            use_causal_mask=True)
        norm1_out = self.layernorm1(attn_out+inputs)
        drop1_out = self.dropout1(norm1_out)
        dense_proj_out = self.dense_proj(drop1_out)
        norm2_out = self.layernorm2(drop1_out+dense_proj_out)
        drop2_out = self.dropout2(norm2_out)
        return drop2_out

In [15]:
inputs = tf.keras.layers.Input(shape=(None,))
embeddings = PositionalEmbedding(sequence_length, vocab_size, EMBED_DIM)(inputs)
x = embeddings
for layer in range(NUM_BLOCKS):
    x = TransformerDecoder(NUM_HEADS, EMBED_DIM, DENSE_DIM, DROPOUT_RATE)(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(vocab_size, activation='linear', kernel_initializer='glorot_uniform')(x)
transformer = tf.keras.models.Model(inputs, output)

In [16]:
transformer.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding            │ (None, None, 384)      │     4,725,888 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder             │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, None, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, None, 12050)    │     4,639,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,914,066 (49.26 MB)

 Trainable params: 12,914,066 (49.26 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
import numpy as np
import tensorflow as tf

def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-9) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)
    
def generate_text(prompt, max_length=50):
    for _ in range(max_length):
        tokenized = vectorizer([prompt])
        preds = transformer(tokenized, training=False)
        next_id = tf.argmax(preds[0, len(prompt.split()) + 1, :]).numpy()
        next_word = vectorizer.get_vocabulary()[next_id]
        prompt += " " + next_word
        if next_id == 0:
            break
    return prompt

text = generate_text('thou shall')
print(text)

thou shall weepst rail elbow servingman fretting would would birthan bows writing whiteliverd would preferrst ripening vincetinos appealld auroras effuse would ribs fawns dauntless federary marchd briefly unaccustomd truth hardly paper brandon hoarding birthan dulcet sweetheart moons ripening hungerford ripening bug complete haver federary lethargy dauntless lethargy fondness shelves uncleanness oermatchd errand


In [18]:
class TextGeneration(tf.keras.callbacks.Callback):
    def __init__(self, text):
        super().__init__()
        self.text = text

    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 50 == 0:
            clear_output(wait=True)
            output = generate_text(self.text)
            print(output)

In [19]:
import tensorflow.keras.backend as K

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=5e-4)

def perplexity(y_true, y_pred):
    cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
    return K.exp(K.mean(cross_entropy))
callback = TextGeneration('to be or not to be')
transformer.compile(loss = loss_fn,
                    metrics = ['accuracy', perplexity],
                    optimizer=opt)
history = transformer.fit(ds,callbacks = [callback], epochs = 400) 

to be or not to be the how that with nor so it and the exeter should not it it have the i when is this but the but and and of it that camillo and he nor that of nor and it and the will so not you and to my i when the to
48/48 ━━━━━━━━━━━━━━━━━━━━ 17s 349ms/step - accuracy: 0.9864 - loss: 0.0550 - perplexity: 1.0565


In [21]:
transformer.save_weights('Shakespeare_decoder.weights.h5')

In [22]:
out = transformer(vectorizer(['Thou shall not']))
preds = tf.argmax(out[0])
for i,j in enumerate(preds): 
    if i > 50:
        break
    next_word = vectorizer.get_vocabulary()[i]
    print(next_word)
    



[UNK]
the
and
to
i
of
my
you
a
that
in
is
not
for
with
it
me
your
be
his
he
this
but
have
as
thou
him
so
what
thy
will
king
by
no
we
all
shall
our
do
if
her
are
lord
thee
now
on
good
from
o
come
